### Prototype code to scrape the top headlines and their corresponding article text from News API

In [1]:
import pandas as pd
import json

In [2]:
import pprint
import requests

In [3]:
# API Key for News API
secret = 'd05c3ae08abd461f90b3dc555af1bb59'

In [4]:
# Define the endpoint to extract all the top headlines 
url = 'https://newsapi.org/v2/top-headlines?'

In [5]:
# Specify the query and number of returns - Limit the headlines to country US, for now
parameters = {
    'language': 'en',
    'country':'us',
    'pageSize': 100,
    'apiKey': secret 
}

In [6]:
categories = ['business','entertainment','general','health','science','sports','technology']


In [7]:
from newspaper import Article
from newspaper import Config


article_title = []
article_authors = []
article_text = []
article_summary = []
article_date = []
article_top_image = []
failed_url = []
category_articles ={}
pages= range(1,2)

In [8]:
count =0
for category in categories:
    name = category
    print(name)
    catDF= None
    parameters['category'] = category
    response = requests.get(url, params=parameters)
    if response.status_code != requests.codes.ok:
        print(f"Bad result : {response.url}")
        continue
    response_json = response.json()

    df = pd.DataFrame.from_dict(response_json)
    df = pd.concat([df.drop(['articles'], axis=1), df['articles'].apply(pd.Series)], axis=1) 

    rows = []
    for index, row in df.iterrows():
        h = row['url']
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
        config = Config()
        config.verbose=True
        config.browser_user_agent = user_agent
        newsarticle = Article(h, config=config)
        try :
            newsarticle.download()
            newsarticle.parse()
            if not newsarticle.text :
                print(f"Unable to parse article: {h}")
                continue 
            artdict = {}                
            artdict['articleText']=newsarticle.text
            for column in df:
                artdict[column]= row[column]
            rows.append(artdict)
            count += 1
            if (count % 50 == 0 ):
                print('Number of articles: '+str(count))

        except :
             print(f'***FAILED TO DOWNLOAD***{newsarticle.url}')
             pass
    if name in category_articles:
        category_articles[name].append(pd.DataFrame(rows),ignore_index=True)
    else:
        category_articles[name]= pd.DataFrame(rows)


print('Total number of articles: '+str(count))
    

business
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9TjllLVFUNUd3NEnSAQA?oc=5
Unable to parse article: http://komonews.com/news/coronavirus/first-seattle-volunteer-in-human-covid-19-vaccine-trials-says-she-feels-fantastic
Number of articles: 50
Unable to parse article: https://www.bizjournals.com/denver/news/2020/07/09/visible-permanent-work-from-home-denver.html
entertainment
Unable to parse article: https://www.tennessean.com/picture-gallery/news/2020/07/10/photos-funeral-country-music-star-charlie-daniels/5409824002/
***FAILED TO DOWNLOAD***https://amp.theguardian.commusic/2020/jul/10/pop-smoke-five-arrested-in-connection-with-brooklyn-rappers-murder-bashar-jackson
Number of articles: 100
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9TGU1SU1HNS1DaEXSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8

In [9]:
#category_articles['business_1']

In [10]:
for cat in category_articles.keys():
    print(cat)
    df = category_articles[cat]
    df.to_json(f'resources/{cat}_top_headline_data.json', orient='index')
    
    

business
entertainment
general
health
science
sports
technology
